In [ ]:
from pathlib import Path
import xarray as xr
import hvplot.xarray              
import hvplot.pandas              
import holoviews as hv
from poseidon.data.schema import load_shard
import numpy as np
import pandas as pd
import yaml


# Load Dependencies and Open File
Inspect the SWOT Expert granule for cycle 2, pass 481 and prepare data for visualization within the configured Gulf of Mexico bounding box.

In [17]:

hv.extension("bokeh")

granule_path = Path("..") / "data" / "sources" / "swot" / "SWOT_L2_LR_SSH_Expert_002_481_20230828T055413_20230828T064455_PGC0_01.nc"
if not granule_path.exists():
    granule_path = Path("data") / "sources" / "swot" / "SWOT_L2_LR_SSH_Expert_002_481_20230828T055413_20230828T064455_PGC0_01.nc"
granule_path = granule_path.resolve()

ds = xr.open_dataset(granule_path)
granule_path

PosixPath('/Users/mako3626/newfrontiers/poseidon/data/sources/swot/SWOT_L2_LR_SSH_Expert_002_481_20230828T055413_20230828T064455_PGC0_01.nc')

# Inspect Dataset Structure
Review dataset metadata, dimensions, and confirm presence of the `ocean_tide_fes` correction alongside position coordinates.

In [18]:
ds

<xarray.Dataset> Size: 400MB
Dimensions:                                (num_lines: 9866, num_pixels: 69,
                                            num_sides: 2)
Coordinates:
    latitude                               (num_lines, num_pixels) float64 5MB ...
    longitude                              (num_lines, num_pixels) float64 5MB ...
    latitude_nadir                         (num_lines) float64 79kB ...
    longitude_nadir                        (num_lines) float64 79kB ...
Dimensions without coordinates: num_lines, num_pixels, num_sides
Data variables: (12/98)
    time                                   (num_lines) datetime64[ns] 79kB ...
    time_tai                               (num_lines) datetime64[ns] 79kB ...
    ssh_karin                              (num_lines, num_pixels) float64 5MB ...
    ssh_karin_qual                         (num_lines, num_pixels) float64 5MB ...
    ssh_karin_uncert                       (num_lines, num_pixels) float64 5MB ...
    ssha_karin                             (num_lines, num_pixels) float64 5MB ...
    ...                                     ...
    swh_ssb_cor_source                     (num_lines, num_pixels) float32 3MB ...
    swh_ssb_cor_source_2                   (num_lines, num_pixels) float32 3MB ...
    wind_speed_ssb_cor_source              (num_lines, num_pixels) float32 3MB ...
    wind_speed_ssb_cor_source_2            (num_lines, num_pixels) float32 3MB ...
    volumetric_correlation                 (num_lines, num_pixels) float64 5MB ...
    volumetric_correlation_uncert          (num_lines, num_pixels) float64 5MB ...
Attributes: (12/62)
    Conventions:                                   CF-1.7
    title:                                         Level 2 Low Rate Sea Surfa...
    institution:                                   JPL
    source:                                        Ka-band radar interferometer
    history:                                       2024-01-19T15:20:07Z : Cre...
    platform:                                      SWOT
    ...                                            ...
    ellipsoid_semi_major_axis:                     6378137.0
    ellipsoid_flattening:                          0.0033528106647474805
    good_ocean_data_percent:                       59.97263311656884
    ssha_variance:                                 2.1124053578647013
    references:                                    V1.2.1
    equator_longitude:                             -93.53

In [19]:
ssh = ds["ocean_tide_fes"]
lat = ds["latitude"]
lon = ds["longitude"]

print(f"ocean_tide_fes dims: {ssh.dims}, shape: {ssh.shape}")
print("ocean_tide_fes attributes:")
for key, value in ssh.attrs.items():
    print(f"  {key}: {value}")

lat_range = (float(lat.min()), float(lat.max()))
lon_range = (float(lon.min()), float(lon.max()))
print(f"Latitude range: {lat_range[0]:.3f} to {lat_range[1]:.3f}")
print(f"Longitude range: {lon_range[0]:.3f} to {lon_range[1]:.3f}")


ocean_tide_fes dims: ('num_lines', 'num_pixels'), shape: (9866, 69)
ocean_tide_fes attributes:
  long_name: geocentric ocean tide height (FES)
  source: FES2014b (Carrere et al., 2016)
  institution: LEGOS/CNES
  units: m
  valid_min: -300000
  valid_max: 300000
  comment: Geocentric ocean tide height. Includes the sum total of the ocean tide, the corresponding load tide (load_tide_fes) and equilibrium long-period ocean tide height (ocean_tide_eq).
Latitude range: -78.272 to 78.272
Longitude range: 182.751 to 350.196


# Prepare Geospatial Data for Mapping
Subset the dataset to the configured bounding box, convert to a dataframe, and clean missing values before visualization.

# Inspect Training Shard Target
Load the cycle 2, pass 481 shard used for training and examine the `y` field that the model attempts to fit.

In [25]:

shard_path = Path("..") / "data" / "shards" / "swot_ssh_single_pass" / "whole_shards" / "shard_c002_p481_SWOT_L2_LR_SSH_Expert_002_481_20230828T055413_20230828T064455_PGC0_01.npz"
if not shard_path.exists():
    shard_path = Path("data") / "shards" / "swot_ssh_single_pass" / "whole_shards" / "shard_c002_p481_SWOT_L2_LR_SSH_Expert_002_481_20230828T055413_20230828T064455_PGC0_01.npz"
shard_path = shard_path.resolve()

shard_data = load_shard(shard_path)
print(f"Loaded shard: {shard_path}")
print("Shard keys:", list(shard_data.keys()))
print("Sample sizes:", {key: shard_data[key].shape for key in shard_data})

Loaded shard: /Users/mako3626/newfrontiers/poseidon/data/shards/swot_ssh_single_pass/whole_shards/shard_c002_p481_SWOT_L2_LR_SSH_Expert_002_481_20230828T055413_20230828T064455_PGC0_01.npz
Shard keys: ['lat', 'lon', 't', 'y', 'cycle', 'pas']
Sample sizes: {'lat': (31366,), 'lon': (31366,), 't': (31366,), 'y': (31366,), 'cycle': (31366,), 'pas': (31366,)}


In [26]:
shard_df = pd.DataFrame(
    {
        "latitude": shard_data["lat"].astype("float32"),
        "longitude": shard_data["lon"].astype("float32"),
        "time": shard_data["t"],
        "target_y": shard_data["y"].astype("float32"),
    }
)

print(shard_df.describe(percentiles=[0.02, 0.5, 0.98]))
shard_df.head()

           latitude     longitude          time      target_y
count  31366.000000  31366.000000  3.136600e+04  31366.000000
mean      25.783897    -89.346809  1.693204e+09      0.227765
std        2.604329      0.647623  5.961465e+01      0.189155
min       21.213724    -90.831993  1.693204e+09     -0.860700
2%        21.451533    -90.569090  1.693204e+09     -0.045170
50%       25.784904    -89.360909  1.693204e+09      0.223500
98%       30.122198    -88.063683  1.693204e+09      0.646540
max       30.391073    -87.756241  1.693204e+09      4.582500


,latitude,longitude,time,target_y
0,21.353317,-90.831993,1.693204e+09,0.3438
1,21.350397,-90.812965,1.693204e+09,0.3914
2,21.347473,-90.793938,1.693204e+09,0.4113
3,21.344549,-90.774910,1.693204e+09,0.4016
4,21.341621,-90.755882,1.693204e+09,0.3916


In [27]:

config_path = Path("..") / "configs" / "data" / "swot_data.yaml"
if not config_path.exists():
    config_path = Path("configs") / "data" / "swot_data.yaml"
with config_path.open() as cfg_file:
    config = yaml.safe_load(cfg_file)
min_lon, min_lat, max_lon, max_lat = config["bbox"]

lat_vals = ds["latitude"].values.astype("float32").ravel()
lon_vals = ds["longitude"].values.astype("float32").ravel()
tide_vals = ds["ocean_tide_fes"].values.astype("float32").ravel()

raw_df = pd.DataFrame(
    {
        "latitude": lat_vals,
        "longitude": lon_vals,
        "ocean_tide_fes": tide_vals,
    }
)

raw_df = raw_df[
    np.isfinite(raw_df["latitude"])
    & np.isfinite(raw_df["longitude"])
    & np.isfinite(raw_df["ocean_tide_fes"])
].copy()

raw_df["longitude_wrapped"] = np.where(
    raw_df["longitude"] > 180,
    raw_df["longitude"] - 360,
    raw_df["longitude"],
).astype("float32")

bbox_mask = (
    (raw_df["longitude_wrapped"] >= min_lon)
    & (raw_df["longitude_wrapped"] <= max_lon)
    & (raw_df["latitude"] >= min_lat)
    & (raw_df["latitude"] <= max_lat)
)

geo_df = (
    raw_df.loc[bbox_mask, ["latitude", "longitude_wrapped", "ocean_tide_fes"]]
    .rename(columns={"longitude_wrapped": "longitude"})
    .reset_index(drop=True)
)

print("Applying bounding box:", (min_lon, min_lat, max_lon, max_lat))
geo_df.head()

Applying bounding box: (-99.0, 17.0, -79.0, 31.0)


,latitude,longitude,ocean_tide_fes
0,17.001135,-91.584686,-0.0058
1,17.018995,-91.581757,-0.0058
2,17.016201,-91.563202,-0.0058
3,17.013405,-91.544647,-0.0058
4,17.010607,-91.526093,-0.0058


In [28]:
print(min_lon, max_lon)

-99.0 -79.0


In [29]:
raw_df["longitude"]-180

10143      14.349930
10144      14.365570
10145      14.381271
10146      14.397003
10147      14.412781
             ...    
680749    170.178741
680750    170.178497
680751    170.178253
680752    170.178009
680753    170.177765
Name: longitude, Length: 670583, dtype: float32

In [30]:
print(min_lat, max_lat)

17.0 31.0


# Visualize ocean_tide_fes on Map
Plot the filtered FES ocean tide correction against latitude and longitude using hvPlot with geospatial tiles and quantile-based color scaling.

In [31]:
vmin, vmax = geo_df["ocean_tide_fes"].quantile([0.02, 0.98]).astype(float)

try:
    map_plot = geo_df.hvplot.scatter(
        x="longitude",
        y="latitude",
        c="ocean_tide_fes",
        cmap="viridis",
        geo=True,
        tiles="EsriOcean",
        colorbar=True,
        clim=(float(vmin), float(vmax)),
        frame_width=700,
        frame_height=450,
        alpha=0.7,
        title="SWOT ocean_tide_fes (Cycle 2 Pass 481)",
        xlim=(min_lon, max_lon),
        ylim=(min_lat, max_lat),
    )
except Exception as exc:
    print("Geographic rendering failed, falling back to plain scatter.")
    print(exc)
    map_plot = geo_df.hvplot.scatter(
        x="longitude",
        y="latitude",
        c="ocean_tide_fes",
        cmap="viridis",
        colorbar=True,
        clim=(float(vmin), float(vmax)),
        frame_width=700,
        frame_height=450,
        alpha=0.7,
        title="SWOT ocean_tide_fes (Cycle 2 Pass 481)",
        xlim=(min_lon, max_lon),
        ylim=(min_lat, max_lat),
    )

map_plot

Geographic rendering failed, falling back to plain scatter.
The `geoviews` package must be installed in order to use geographic features. Install it with pip or conda.


:Scatter   [longitude]   (latitude,ocean_tide_fes)